In [1]:
import pandas as pd


df = pd.read_csv('../data/curated/engineered-data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/curated/engineered-data.csv'

In [ ]:
df = df[df['bedrooms'] >= 2]
df = df[df['bedrooms'] <= 4]
df.columns


Index(['suburb', 'postcode', 'price', 'propertyType', 'bedrooms', 'bathrooms',
       'parkingSpaces', 'studies', 'furnished', 'closest_primary_distance',
       'closest_secondary_distance', 'closest_train_distance',
       'closest_tram_distance', 'closest_bus_distance',
       'closest_park_distance', 'min_pri_icsea', 'min_sec_icsea',
       'park_walkability', 'is_primary_school_walkable',
       'is_secondary_school_walkable', 'is_tram_stop_walkable'],
      dtype='object')

Affordability

In [ ]:
dfaff = df.groupby('suburb').agg({'price':'mean','bedrooms':'mean'}).reset_index()
dfaff['Affordability'] = dfaff['price'] / dfaff['bedrooms']
dfaff = dfaff.sort_values(by='Affordability')

In [ ]:
dfaff['suburb'][0:5]

49            Burnley
4         Albert Park
104    East Melbourne
253         Southbank
96          Docklands
Name: suburb, dtype: object

Liveability

In [ ]:
df['overall_icsea']=df['min_pri_icsea']+df['min_sec_icsea']

In [ ]:
icsea_list=[df['overall_icsea'].quantile(0.33),df['overall_icsea'].quantile(0.66)]

In [ ]:
crime_list=[df['crimenumber'].quantile(0.33),df['overall_icsea'].quantile(0.66)]

In [ ]:
def calculate_livability(row):
    sum = 0
    # park walkability
    if row['park_walkability'] == '$<$1.5 km':
        sum+=1
    elif row['park_walkability'] == '1.5-5km':
        sum += 0.5
    # School walkbility
    if row['is_primary_school_walkable'] == 'Yes': sum += 0.5
    if row['is_secondary_school_walkable'] == 'Yes': sum += 0.5
    # Public transport walkbility
    if row['is_tram_stop_walkable'] == 'Yes': sum += 1
    # School rating
    if row['overall_icsea'] > icsea_list[1]:
        sum+=1
    elif row['overall_icsea'] > icsea_list[0]:
        sum+=0.5
    # Crime number
    if row['crimenumber'] < crime_list[0]:
        sum+=1
    elif row['crimenumber'] < crime_list[1]:
        sum+=0.5

    return sum


In [ ]:
df['livability']=df.apply(calculate_livability, axis=1)

In [ ]:
dfliv = df.groupby('suburb').mean('livability').reset_index()
dfliv = dfliv.sort_values(by='livability',ascending=False)

'Abbotsford'

In [ ]:
dfliv['suburb'][0:5]

4          Albert Park
232           Richmond
227     Port Melbourne
300            Windsor
250    South Melbourne
Name: suburb, dtype: object